# Manejar valores nulos

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_csv("/home/miguel/Desktop/ML_Notebooks_&_Projects/30_Dias/Dia_9/First_model/melb_data.csv")
nd = data.drop(['Price'], axis = 1)
X = nd.select_dtypes(exclude=['object'])
y = data.Price
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

## Creamos una funcion para testear el dataset:

In [17]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators = 10, random_state = 0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

## 1. Quitar columnas con valores nulos

In [18]:
cols_missing = [col for col in X_train.columns if X_train[col].isnull().sum().any()]

NX_train = X_train.drop(cols_missing, axis = 1)
NX_valid = X_valid.drop(cols_missing, axis = 1)

print("MAE para el primer metodo:\n", score_dataset(NX_train, NX_valid, y_train, y_valid))

MAE para el primer metodo:
 183550.22137772635


## 2. Suplantar el valor nulo con el promedio de la columna

In [19]:
from sklearn.impute import SimpleImputer
# Imputacion
imputer = SimpleImputer()
NX_train = pd.DataFrame(imputer.fit_transform(X_train))
NX_valid = pd.DataFrame(imputer.transform(X_valid))

# Devolverle los nombres a las columnas
NX_train.columns = X_train.columns
NX_valid.columns = X_valid.columns

# Resultados
print("MAE para el segundo metodo:\n", score_dataset(NX_train, NX_valid, y_train, y_valid))

MAE para el segundo metodo:
 178166.46269899711


## 3. Suplantar el valor nulo y mantener la informacion de la perdida

In [20]:
X_trainp = X_train.copy()
X_validp = X_valid.copy()

# Crear la nueva columna
for col in cols_missing:
    X_trainp[col + "_was_missing"] = X_trainp[col].isnull()
    X_validp[col + "_was_missing"] = X_validp[col].isnull()

#Imputar
imputer = SimpleImputer()
NX_train = pd.DataFrame(imputer.fit_transform(X_trainp))
NX_valid = pd.DataFrame(imputer.transform(X_validp))

#Devolver columnas
NX_train.columns = X_trainp.columns
NX_valid.columns = X_validp.columns

#Resultados
print("MAE para el tercer metodo:\n", score_dataset(NX_train, NX_valid, y_train, y_valid))

MAE para el tercer metodo:
 178927.503183954


# Conclusiones
La segunda funciona mejor debido a que la columna que añadimos en el tercer metodo no añadia valor estadistico al modelo, pero ambas funcionan mejor que eliminar la columna.